In [11]:
from pathlib import Path
import pandas as pd

# ============================
# 1) Chemin du dossier CSV
# ============================

# Si ton notebook est dans "C:/Users/Lenovo/Desktop/feryeni/"
# et le dossier CSV est "C:/Users/Lenovo/Desktop/feryeni/CSV",
# alors ça suffit :
DATA_DIR = Path("CSV")

# Sinon, mets le chemin complet :
# DATA_DIR = Path(r"C:\Users\Lenovo\Desktop\feryeni\CSV")


def load_csv(filename: str) -> pd.DataFrame:
    """
    Charge un CSV depuis DATA_DIR avec un message clair si le fichier n'existe pas.
    """
    path = DATA_DIR / filename
    if not path.exists():
        raise FileNotFoundError(f"❌ Fichier introuvable : {path}")
    print(f"✅ Chargement de {path}")
    return pd.read_csv(path)


# ============================
# 2) Chargement des fichiers
# ============================

matches_gps   = load_csv("matchs_gps.csv")
training_gps  = load_csv("training_gps.csv")
wys_matches   = load_csv("wyscout_matchs.csv")
wys_gk        = load_csv("wyscout_players_goalkeeper.csv")
wys_outfield  = load_csv("wyscout_players_outfield.csv")





✅ Chargement de CSV\matchs_gps.csv
✅ Chargement de CSV\training_gps.csv
✅ Chargement de CSV\wyscout_matchs.csv
✅ Chargement de CSV\wyscout_players_goalkeeper.csv
✅ Chargement de CSV\wyscout_players_outfield.csv


In [12]:
from pathlib import Path
import pandas as pd
import numpy as np
import re


In [21]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

# =====================================================
# 0) Chemins & chargement des CSV bruts
# =====================================================

BASE_DIR = Path("CSV")
RAW_DIR = BASE_DIR          # les CSV bruts sont directement dans CSV
OUT_DIR = BASE_DIR / "star_schema"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("Lecture des CSV bruts...")

matches_gps   = pd.read_csv(RAW_DIR / "matchs_gps.csv", low_memory=False)
training_gps  = pd.read_csv(RAW_DIR / "training_gps.csv", low_memory=False)
wys_matches   = pd.read_csv(RAW_DIR / "wyscout_matchs.csv", low_memory=False)
wys_gk        = pd.read_csv(RAW_DIR / "wyscout_players_goalkeeper.csv", low_memory=False)
wys_outfield  = pd.read_csv(RAW_DIR / "wyscout_players_outfield.csv", low_memory=False)

print("✅ Chargement terminé")
print("matches_gps       :", matches_gps.shape)
print("training_gps      :", training_gps.shape)
print("wyscout_matchs    :", wys_matches.shape)
print("wyscout_GK        :", wys_gk.shape)
print("wyscout_outfield  :", wys_outfield.shape)


# =====================================================
# 1) Fonctions utilitaires
# =====================================================

def slugify(col: str) -> str:
    """
    Nettoie un nom de colonne en snake_case.
    Utile surtout pour les tables GPS.
    """
    col = str(col).strip().lower()
    col = col.replace('%', 'pct').replace('#', 'num')
    col = re.sub(r'[^0-9a-z]+', '_', col)
    col = re.sub(r'_+', '_', col).strip('_')
    return col


def parse_date_col(s):
    """Parse une colonne de dates en datetime (jour/mois ou iso)."""
    return pd.to_datetime(s, dayfirst=True, errors="coerce")


def make_match_uid(df: pd.DataFrame) -> pd.Series:
    """
    Identifiant de match cohérent basé sur (match, date_parsed, competition).

    - On parse la date puis on la formate en 'YYYY-MM-DD'
    - On strip les espaces sur match / competition
    => même clé pour toutes les tables qui ont (match, date, competition).
    """
    date_parsed = parse_date_col(df["date"])
    date_str = date_parsed.dt.strftime("%Y-%m-%d")

    match_str = df["match"].astype(str).str.strip()
    comp_str  = df["competition"].astype(str).str.strip()

    return match_str + "|" + date_str + "|" + comp_str


# =====================================================
# 2) DIM TEAM
# =====================================================

teams_gps = pd.concat(
    [matches_gps["team_name"], training_gps["team_name"]],
    ignore_index=True,
)

teams_wys = pd.concat(
    [wys_matches["team_name"], wys_gk["team_name"], wys_outfield["team_name"]],
    ignore_index=True,
)

all_team_names = (
    pd.concat([teams_gps, teams_wys], ignore_index=True)
      .dropna()
      .drop_duplicates()
      .sort_values()
      .reset_index(drop=True)
)

dim_team = pd.DataFrame({"team_name_std": all_team_names})
dim_team["team_key"] = np.arange(1, len(dim_team) + 1)

dim_team["team_name_gps"] = np.where(
    dim_team["team_name_std"].isin(teams_gps.dropna().unique()),
    dim_team["team_name_std"],
    pd.NA,
)
dim_team["team_name_wyscout"] = np.where(
    dim_team["team_name_std"].isin(teams_wys.dropna().unique()),
    dim_team["team_name_std"],
    pd.NA,
)

dim_team["image_url"]        = pd.NA
dim_team["image_url_no_bg"]  = pd.NA
dim_team["image_source"]     = pd.NA
dim_team["created_at"]       = pd.Timestamp.utcnow().normalize()

dim_team = dim_team[
    [
        "team_key",
        "team_name_std",
        "team_name_gps",
        "team_name_wyscout",
        "image_url",
        "image_url_no_bg",
        "image_source",
        "created_at",
    ]
]

team_key_by_name = dict(zip(dim_team["team_name_std"], dim_team["team_key"]))


# =====================================================
# 3) DIM PLAYER
# =====================================================

# noms depuis Wyscout (GK + outfield)
wys_players_raw = pd.concat(
    [
        wys_gk[["player", "team_name"]],
        wys_outfield[["player", "team_name"]],
    ],
    ignore_index=True,
).dropna(subset=["player"])

wys_players_raw = wys_players_raw.drop_duplicates(subset=["player"])

# noms depuis GPS
gps_players_raw = pd.concat(
    [
        matches_gps[["name", "team_name"]].rename(columns={"name": "player"}),
        training_gps[["name", "team_name"]].rename(columns={"name": "player"}),
    ],
    ignore_index=True,
).dropna(subset=["player"])

players_all = pd.concat([wys_players_raw, gps_players_raw], ignore_index=True)
players_all = (
    players_all.sort_values(["player", "team_name"])
               .drop_duplicates(subset=["player"], keep="first")
               .reset_index(drop=True)
)

dim_player = pd.DataFrame()
dim_player["player_key"]         = np.arange(1, len(players_all) + 1)
dim_player["player_name_std"]    = players_all["player"]
dim_player["player_name_gps"]    = np.where(
    dim_player["player_name_std"].isin(gps_players_raw["player"].unique()),
    dim_player["player_name_std"],
    pd.NA,
)
dim_player["player_name_wyscout"] = np.where(
    dim_player["player_name_std"].isin(wys_players_raw["player"].unique()),
    dim_player["player_name_std"],
    pd.NA,
)
dim_player["player_name_sportdb"] = pd.NA

dim_player["team_key"] = players_all["team_name"].map(team_key_by_name)

dim_player["image_url"]        = pd.NA
dim_player["image_url_no_bg"]  = pd.NA
dim_player["image_source"]     = pd.NA
dim_player["created_at"]       = pd.Timestamp.utcnow().normalize()

dim_player = dim_player[
    [
        "player_key",
        "player_name_std",
        "player_name_gps",
        "player_name_wyscout",
        "player_name_sportdb",
        "team_key",
        "image_url",
        "image_url_no_bg",
        "image_source",
        "created_at",
    ]
]

player_key_by_name = dict(zip(dim_player["player_name_std"], dim_player["player_key"]))


# =====================================================
# 4) DIM COMPETITION
# =====================================================

competitions = pd.concat(
    [wys_matches["competition"], wys_gk["competition"], wys_outfield["competition"]],
    ignore_index=True,
).dropna().drop_duplicates().sort_values()

dim_competition = pd.DataFrame({"competition_name_std": competitions})
dim_competition["competition_key"] = np.arange(1, len(dim_competition) + 1)

dim_competition["competition_name_gps"]      = pd.NA
dim_competition["competition_name_wyscout"]  = dim_competition["competition_name_std"]
dim_competition["competition_name_sportdb"]  = pd.NA
dim_competition["created_at"]                = pd.Timestamp.utcnow().normalize()

dim_competition = dim_competition[
    [
        "competition_key",
        "competition_name_std",
        "competition_name_gps",
        "competition_name_wyscout",
        "competition_name_sportdb",
        "created_at",
    ]
]

comp_key_by_name = dict(
    zip(dim_competition["competition_name_std"], dim_competition["competition_key"])
)


# =====================================================
# 5) DIM DATE
# =====================================================

all_dates_raw = pd.concat(
    [
        matches_gps["date"],
        training_gps["date"],
        wys_matches["date"],
        wys_gk["date"],
        wys_outfield["date"],
    ],
    ignore_index=True,
)

all_dates = parse_date_col(all_dates_raw)
all_dates = (
    all_dates.dropna()
    .drop_duplicates()
    .sort_values()
    .reset_index(drop=True)
)

dim_date = pd.DataFrame({"date": all_dates})
dim_date["date_key"]     = dim_date["date"].dt.strftime("%Y%m%d").astype(int)
dim_date["year"]         = dim_date["date"].dt.year
dim_date["quarter"]      = dim_date["date"].dt.quarter
dim_date["month"]        = dim_date["date"].dt.month
dim_date["month_name"]   = dim_date["date"].dt.month_name()
dim_date["week_of_year"] = dim_date["date"].dt.isocalendar().week.astype(int)
dim_date["day"]          = dim_date["date"].dt.day
dim_date["day_of_week"]  = dim_date["date"].dt.weekday + 1
dim_date["day_name"]     = dim_date["date"].dt.day_name()
dim_date["is_weekend"]   = dim_date["day_of_week"].isin([6, 7])

year  = dim_date["year"]
month = dim_date["month"]
dim_date["season"] = np.where(
    month >= 7,
    year.astype(str) + "-" + (year + 1).astype(str),
    (year - 1).astype(str) + "-" + year.astype(str),
)

dim_date["created_at"] = pd.Timestamp.utcnow().normalize()

dim_date = dim_date[
    [
        "date_key",
        "date",
        "year",
        "quarter",
        "month",
        "month_name",
        "week_of_year",
        "day",
        "day_of_week",
        "day_name",
        "is_weekend",
        "season",
        "created_at",
    ]
]

date_key_by_date = dict(zip(dim_date["date"], dim_date["date_key"]))


# =====================================================
# 6) FACT PLAYER WYSCOUT (construit AVANT dim_match)
#    => puis dim_match déduite des players
# =====================================================

wys_gk["position_type"]       = "GK"
wys_outfield["position_type"] = "OUTFIELD"

players_wys_all = pd.concat([wys_gk, wys_outfield], ignore_index=True, sort=False)

players_wys_all["match_date"] = parse_date_col(players_wys_all["date"])
players_wys_all["date_key"]   = players_wys_all["match_date"].map(date_key_by_date)
players_wys_all["team_key"]   = players_wys_all["team_name"].map(team_key_by_name)
players_wys_all["player_key"] = players_wys_all["player"].map(player_key_by_name)
players_wys_all["competition_key"] = players_wys_all["competition"].map(comp_key_by_name)

# match_uid basé SUR LES PLAYERS
players_wys_all["match_uid"] = make_match_uid(players_wys_all)

# =====================================================
# 7) DIM MATCH basé sur players_wys_all, enrichi par wys_matches
# =====================================================

# base : toutes les combinaisons (match, date, competition) qu'on voit chez les joueurs
match_dim_raw = (
    players_wys_all[["match_uid", "match_date", "match", "competition"]]
    .drop_duplicates(subset=["match_uid"])
    .reset_index(drop=True)
)

# essayer d'enrichir avec score / equipe / adversaire depuis wys_matches
wys_matches = wys_matches.copy()
wys_matches["match_uid"] = make_match_uid(wys_matches)

match_extra = (
    wys_matches[["match_uid", "score", "equipe", "adversaire"]]
    .drop_duplicates(subset=["match_uid"])
)

match_dim_raw = match_dim_raw.merge(
    match_extra,
    on="match_uid",
    how="left"
)

dim_match = pd.DataFrame()
dim_match["match_key"]       = np.arange(1, len(match_dim_raw) + 1)
dim_match["match_date"]      = match_dim_raw["match_date"]
dim_match["competition_key"] = match_dim_raw["competition"].map(comp_key_by_name)
dim_match["date_key"]        = match_dim_raw["match_date"].map(date_key_by_date)
dim_match["score"]           = match_dim_raw["score"]

# home/away si on a pu les récupérer (sinon NaN)
dim_match["home_team_key"] = match_dim_raw["equipe"].map(team_key_by_name)
dim_match["away_team_key"] = match_dim_raw["adversaire"].map(team_key_by_name)

# saison depuis dim_date
dim_match = dim_match.merge(
    dim_date[["date_key", "season"]],
    on="date_key",
    how="left",
)
dim_match.rename(columns={"season": "season"}, inplace=True)

gps_match_dates = parse_date_col(matches_gps["date"]).dropna().unique()
dim_match["has_gps_data"]     = dim_match["match_date"].isin(gps_match_dates)
dim_match["has_wyscout_data"] = True
dim_match["has_sportdb_data"] = False
dim_match["created_at"]       = pd.Timestamp.utcnow().normalize()

dim_match = dim_match[
    [
        "match_key",
        "match_date",
        "home_team_key",
        "away_team_key",
        "competition_key",
        "date_key",
        "score",
        "season",
        "has_gps_data",
        "has_wyscout_data",
        "has_sportdb_data",
        "created_at",
    ]
]

# mapping match_uid -> match_key basé sur LES PLAYERS
match_key_by_uid = dict(zip(match_dim_raw["match_uid"], dim_match["match_key"]))

# maintenant on peut remplir match_key dans players_wys_all
players_wys_all["match_key"] = players_wys_all["match_uid"].map(match_key_by_uid)

print("\n[DEBUG] Proportion de match_key NaN dans players_wys_all :",
      players_wys_all["match_key"].isna().mean())

fact_player_wyscout = players_wys_all.reset_index(drop=True)
fact_player_wyscout.insert(0, "fact_player_wyscout_key", fact_player_wyscout.index + 1)
fact_player_wyscout = fact_player_wyscout.drop(columns=["match_uid"])


# =====================================================
# 8) FACT MATCH WYSCOUT (agrégé à partir de wys_matches)
# =====================================================

fact_match_wyscout = wys_matches.copy()

fact_match_wyscout["match_date"] = parse_date_col(fact_match_wyscout["date"])
fact_match_wyscout["date_key"]   = fact_match_wyscout["match_date"].map(date_key_by_date)
fact_match_wyscout["team_key"]   = fact_match_wyscout["team_name"].map(team_key_by_name)
fact_match_wyscout["opponent_team_key"] = fact_match_wyscout["adversaire"].map(team_key_by_name)
fact_match_wyscout["competition_key"]   = fact_match_wyscout["competition"].map(comp_key_by_name)

fact_match_wyscout["match_uid"] = make_match_uid(fact_match_wyscout)
fact_match_wyscout["match_key"] = fact_match_wyscout["match_uid"].map(match_key_by_uid)

fact_match_wyscout = fact_match_wyscout.reset_index(drop=True)
fact_match_wyscout.insert(0, "fact_match_wyscout_key", fact_match_wyscout.index + 1)
fact_match_wyscout = fact_match_wyscout.drop(columns=["match_uid"])

print("[DEBUG] Proportion de match_key NaN dans fact_match_wyscout :",
      fact_match_wyscout["match_key"].isna().mean())


# =====================================================
# 9) FACT PLAYER GPS (match + training)
# =====================================================

matches_gps2  = matches_gps.copy()
training_gps2 = training_gps.copy()

matches_gps2["session_type"]  = "match"
training_gps2["session_type"] = "training"

gps_all = pd.concat([matches_gps2, training_gps2], ignore_index=True, sort=False)

gps_all["session_date"] = parse_date_col(gps_all["date"])
gps_all["date_key"]     = gps_all["session_date"].map(date_key_by_date)
gps_all["team_key"]     = gps_all["team_name"].map(team_key_by_name)
gps_all["player_key"]   = gps_all["name"].map(player_key_by_name)

# pour l'instant, pas de lien direct avec dim_match
gps_all["match_key"] = pd.NA

fact_player_gps = gps_all.reset_index(drop=True)
fact_player_gps.insert(0, "fact_player_gps_key", fact_player_gps.index + 1)

protected_cols = {
    "fact_player_gps_key",
    "player_key",
    "team_key",
    "match_key",
    "date_key",
    "session_type",
    "session_date",
}
gps_cols_renamed = {
    c: slugify(c)
    for c in fact_player_gps.columns
    if c not in protected_cols
}
fact_player_gps = fact_player_gps.rename(columns=gps_cols_renamed)


# =====================================================
# 10) Sauvegarde des CSV star_schema
# =====================================================

dim_team.to_csv(OUT_DIR / "dim_team.csv", index=False)
dim_player.to_csv(OUT_DIR / "dim_player.csv", index=False)
dim_competition.to_csv(OUT_DIR / "dim_competition.csv", index=False)
dim_date.to_csv(OUT_DIR / "dim_date.csv", index=False)
dim_match.to_csv(OUT_DIR / "dim_match.csv", index=False)

fact_match_wyscout.to_csv(OUT_DIR / "fact_match_wyscout.csv", index=False)
fact_player_wyscout.to_csv(OUT_DIR / "fact_player_wyscout.csv", index=False)
fact_player_gps.to_csv(OUT_DIR / "fact_player_gps.csv", index=False)

print("\n✅ Fichiers star_schema écrits dans :", OUT_DIR)


Lecture des CSV bruts...
✅ Chargement terminé
matches_gps       : (2108, 812)
training_gps      : (7903, 810)
wyscout_matchs    : (9222, 110)
wyscout_GK        : (15323, 20)
wyscout_outfield  : (141558, 72)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1852\903293105.py:49: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, dayfirst=True, errors="coerce")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1852\903293105.py:49: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, dayfirst=True, errors="coerce")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1852\903293105.py:49: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, dayfirst=True, errors="coerce")



[DEBUG] Proportion de match_key NaN dans players_wys_all : 0.0


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1852\903293105.py:49: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, dayfirst=True, errors="coerce")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1852\903293105.py:49: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, dayfirst=True, errors="coerce")


[DEBUG] Proportion de match_key NaN dans fact_match_wyscout : 0.3083929733246584


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1852\903293105.py:432: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  gps_all["session_date"] = parse_date_col(gps_all["date"])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1852\903293105.py:433: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  gps_all["date_key"]     = gps_all["session_date"].map(date_key_by_date)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1852\903293105.py:434: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` ma


✅ Fichiers star_schema écrits dans : CSV\star_schema


In [15]:
from pathlib import Path

DATA_DIR = Path("CSV")          # si tes fichiers sont dans ./CSV
OUT_DIR = DATA_DIR / "star_schema"
OUT_DIR.mkdir(exist_ok=True)

print("OUT_DIR =", OUT_DIR)


OUT_DIR = CSV\star_schema


In [22]:
from pathlib import Path
import pandas as pd

OUT_DIR = Path("CSV") / "star_schema"
fpw = pd.read_csv(OUT_DIR / "fact_player_wyscout.csv", low_memory=False)

print("Proportion de match_key NaN :", fpw["match_key"].isna().mean())
print(fpw[["match", "date", "competition", "match_key"]].head(10))


Proportion de match_key NaN : 0.0
                                       match        date  \
0                  Aston Villa - Chelsea 1:3  2021-12-26   
1          Benfica U23 - Sporting CP U23 1:1  2018-12-04   
2                          Nice - Rennes 1:1  2020-01-24   
3       Nice II - Olympique Marseille II 2:1  2016-04-30   
4                 Sporting CP - Boavista 2:0  2020-02-23   
5  Paços de Ferreira - Vitória Guimarães 0:1  2022-10-08   
6          Vitória Guimarães - Famalicão 3:2  2022-10-31   
7                     Montpellier - Nice 0:0  2023-11-10   
8                      Sporting CP - PSV 4:0  2019-11-28   
9                 Chelsea U19 - Roma U19 0:2  2017-10-18   

                       competition  match_key  
0          England. Premier League          1  
1  Portugal. Liga Revelação Sub 23          2  
2                  France. Ligue 1          3  
3               France. National 2          4  
4          Portugal. Primeira Liga          5  
5          Portug